# Hugging Face
## Transformers
- Transformers provides thousands of pretrained models to perform tasks on different modalities such as text, vision, and audio.
- Text, for tasks like text classification, information extraction, question answering, summarization, translation, and text generation, in over 100 languages.
- Images, for tasks like image classification, object detection, and segmentation.
- Audio, for tasks like speech recognition and audio classification.

# Pipeline Function
The pipeline function returns and end to end object that performers an NLP task on one or several texts
+ There are three main steps involved when you pass some text to a pipeline:

1. The text is preprocessed into a format the model can understand.
2. The preprocessed inputs are passed to the model.
3. The predictions of the model are post-processed, so you can make sense of them.
### Some of the functions are:
+ feature-extraction (get the vector representation of a text)
+ fill-mask
ner (named entity recognition)
+ question-answering
+ sentiment-analysis
+ summarization
+ text-generation
+ translation
+ zero-shot-classification

In [ ]:
from transformers import pipeline

In [ ]:
classifier = pipeline("zero-shot-classification")
classifier(
    "The pipeline function is from HuggingFace transformers",
    candidate_labels = ["education","business","entertainment"]
)

We can also specify model is needed

In [ ]:
generator = pipeline("text-generation",model='distilgpt2')
generator(
    "Machine Learning is the ",
    max_length=20,
    num_return_sequences=3,
)

In [ ]:
classifier = pipeline('sentiment-analysis')
classifier(
    [
        "I have been learning about transformers and it is amazing",
        "I am motivated to use it"
    ]
)

### Preprocessing with a tokenizer
Like other neural networks, Transformer models can’t process raw text directly, so the first step of our pipeline is to convert the text inputs into numbers that the model can make sense of. To do this we use a tokenizer, which will be responsible for:

1. Splitting the input into words, subwords, or symbols (like punctuation) that are called tokens
2. Mapping each token to an integer
3. Adding additional inputs that may be useful to the model
- All this preprocessing needs to be done in exactly the same way as when the model was pretrained
- To do this, we use the **`AutoTokenizer`** class and its **`from_pretrained()`** method.

Note:**Transformer models only accept tensors as input**

In [ ]:
# We can choose the model from Model_Hub from the hugging face
from transformers import AutoTokenizer
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
raw_inputs =     [
        "I have been learning about transformers and it is amazing",
        "I am motivated to use it"
    ]
# Add padding=True and truncation=True to ensure all sequences have the same length
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors='pt')
print(inputs)

We can download our pretrained model the same way we did with our tokenizer. Transformers provides an **`AutoModel`** class which also has a **`from_pretrained()`** method:

In [ ]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

In [ ]:
outputs = model(**inputs)
print(outputs)

- 'input_ids' → Encoded tokenized text
- 'attention_mask' → Tells the model which tokens to pay attention to (useful for padding)
**inputs unpacks the dictionary
When calling model(**inputs), it's equivalent to:
```python
model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])
```
So,
- tokenizer() returns a dictionary of tensors (input_ids, attention_mask, etc.).
- **inputs unpacks the dictionary into individual arguments.
- Hugging Face models expect named arguments instead of a single dictionary.

## Hugging Face Model Classes – When to Use What?

| Model Type | Use Case | Returns |
|------------|---------|---------|
| `AutoModel` | Get hidden states/embeddings | Hidden states (last layer output) |
| `AutoModelForSequenceClassification` | Sentiment analysis, topic classification | Logits for whole sentence |
| `AutoModelForTokenClassification` | NER, POS tagging | Logits per token |
| `AutoModelForQuestionAnswering` | Extractive QA (find answer in text) | Start/end logits |
| `AutoModelForCausalLM` | Text generation (GPT-style) | Next-token logits |
| `AutoModelForMaskedLM` | Fill-in-the-blank (BERT-style) | Logits per token |
| `AutoModelForMultipleChoice` | Multiple-choice QA (exams, tests) | Logits per choice |

 **Quick Tip:**  
- Use `AutoModel` for embeddings.  
- Use `ForSequenceClassification` for sentiment analysis.  
- Use `ForCausalLM` for text generation.  
- Use `ForQuestionAnswering` for extractive QA.  
- Use `ForMaskedLM` for fill-in-the-blank tasks.  



In [ ]:
from transformers import AutoModelForSequenceClassification
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

In [ ]:
print(outputs)

In [ ]:
print(outputs.logits)

Those are not probabilities but logits, the raw, unnormalized scores outputted by the last layer of the model. To be converted to probabilities, they need to go through a SoftMax layer

In [ ]:
import torch
pred = torch.nn.functional.softmax(outputs.logits)
print(pred)

In [ ]:
model.config.id2label